In [1]:
import sys, os

import numpy as np
import pandas as pd

from CART import *
from Utils.plotting import  *
from scipy.stats import norm as ndist
import joblib

# For tree-values
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector

# Select a CRAN mirror to download from
utils = rpackages.importr('utils')
utils.chooseCRANmirror(ind=1)  # Select the first mirror

# Install 'remotes' if it's not already installed
if not rpackages.isinstalled('remotes'):
    utils.install_packages(StrVector(('remotes',)))

import rpy2.robjects as ro

from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects import numpy2ri

In [2]:
# Run the GitHub installation command for 'treevalues'
ro.r('remotes::install_github("anna-neufeld/treevalues")')
ro.r('library(treevalues)')
ro.r('library(rpart)')

R[write to console]: Using GitHub PAT from the git credential store.

R[write to console]: Skipping install of 'treevalues' from a github remote, the SHA1 (55573782) has not changed since last install.
  Use `force = TRUE` to force installation



In [3]:
def generate_test(mu, sd_y):
    n = mu.shape[0]
    return mu + np.random.normal(size=(n,), scale=sd_y)

# Tree-values inference

In [4]:
def tree_values_inference(X, y, mu, sd_y, max_depth=5, level=0.1,
                          X_test=None):
    # Convert the NumPy matrix to an R matrix
    X_r = numpy2ri.py2rpy(X)
    y_r = numpy2ri.py2rpy(y)

    # Assign the R matrix to a variable in the R environment (optional)
    ro.globalenv['X_r'] = X_r
    ro.globalenv['y_r'] = y_r
    ro.globalenv['p'] = X.shape[1]

    # Construct dataset
    ro.r('data <- cbind(y_r, X_r)')
    # Set the column names to "y", "x1", "x2", ..., "x10"
    ro.r('colnames(data) <- c("y", paste0("x", 1:p))')
    ro.r('data = data.frame(data)')

    # Define the rpart tree model
    tree_cmd = ('bls.tree <- rpart(y ~ ., data=data, model = TRUE, ' +
                'control = rpart.control(cp=0.00, minsplit = 50, minbucket = 20, maxdepth=') + str(max_depth) + '))'
    ro.r(tree_cmd)
    bls_tree = ro.r('bls.tree')
    # Plot the tree values (this will plot directly if you have a plotting backend set up)
    # ro.r('treeval.plot(bls.tree, inferenceType=0)')

    # ro.r('print(row.names(bls.tree$frame)[bls.tree$frame$var == "<leaf>"])')
    ro.r('leaf_idx <- (row.names(bls.tree$frame)[bls.tree$frame$var == "<leaf>"])')
    leaf_idx = ro.r['leaf_idx']

    # Get node mapping
    ro.r('idx_full <- 1:nrow(bls.tree$frame)')
    ro.r('mapped_idx <- idx_full[bls.tree$frame$var == "<leaf>"]')

    len = []
    coverage = []
    len_naive = []
    coverage_naive = []

    for i, idx in enumerate(leaf_idx):
        # Get the branch information for a specific branch in the tree
        command = 'branch <- getBranch(bls.tree, ' + str(idx) + ')'
        ro.r(command)
        # Perform branch inference
        ro.r(f'result <- branchInference(bls.tree, branch, type="reg", alpha = 0.10, sigma_y={sd_y})')
        # Get confidence intervals
        confint = ro.r('result$confint')
        len.append(confint[1] - confint[0])

        target_cmd = "contrast <- (bls.tree$where == mapped_idx[" + str(i + 1) + "])"
        ro.r(target_cmd)
        contrast = ro.r('contrast')
        contrast = np.array(contrast)

        contrast = np.array(contrast * 1 / np.sum(contrast))

        target = contrast.dot(mu)
        coverage.append(target >= confint[0] and target <= confint[1])

        # Naive after tree value
        # Confidence intervals
        naive_CI = [contrast.dot(y) -
                    np.linalg.norm(contrast) * sd_y * ndist.ppf(1 - level / 2),
                    contrast.dot(y) +
                    np.linalg.norm(contrast) * sd_y * ndist.ppf(1 - level / 2)]
        coverage_naive.append((target >= naive_CI[0] and target <= naive_CI[1]))
        len_naive.append(naive_CI[1] - naive_CI[0])

    if X_test is not None:
        X_test_r = numpy2ri.py2rpy(X_test)
        ro.globalenv['X_test_r'] = X_test_r
        ro.r('pred <- predict(bls.tree, data = X_test_r)')
        pred = ro.r['pred']
    else:
        pred = None

    return (np.mean(coverage), np.mean(len),
            np.mean(coverage_naive), np.mean(len_naive), pred)

# RRT inference

In [5]:
def randomized_inference(reg_tree, sd_y, y, mu, level=0.1):
    # print(reg_tree.terminal_nodes)
    coverage_i = []
    lengths_i = []

    for node in reg_tree.terminal_nodes:
        pval, dist, contrast, norm_contrast, obs_tar, logW, suff, sel_probs \
            = (reg_tree.condl_node_inference(node=node,
                                             ngrid=10000,
                                             ncoarse=50,
                                             grid_w_const=5,
                                             reduced_dim=None,
                                             sd=sd_y,
                                             use_cvxpy=True))
        target = contrast.dot(mu)

        # This is an interval for
        # eta_*'mu = eta'mu / (norm(eta) * sd_y)
        selective_CI = (dist.equal_tailed_interval(observed=norm_contrast.dot(y),
                                                   alpha=level))
        selective_CI = np.array(selective_CI)
        selective_CI *= np.linalg.norm(contrast) * sd_y
        coverage_i.append((target >= selective_CI[0] and target <= selective_CI[1]))
        lengths_i.append(selective_CI[1] - selective_CI[0])

    return coverage_i, lengths_i

# Inference with UV decomposition

In [6]:
def UV_decomposition(X, y, mu, sd_y,
                     max_depth=5, min_prop=0, min_sample=10, min_bucket=5,
                     level=0.1, gamma=1,
                     X_test=None):
    n = X.shape[0]
    W = np.random.normal(loc=0, scale=sd_y * np.sqrt(gamma), size=(n,))
    U = y + W
    V = y - W / gamma
    sd_V = sd_y * np.sqrt(1 + 1 / gamma)
    reg_tree = RegressionTree(min_samples_split=min_sample, max_depth=max_depth,
                              min_proportion=min_prop, min_bucket=min_bucket)
    reg_tree.fit(X, U, sd=0)

    coverage = []
    lengths = []

    for node in reg_tree.terminal_nodes:
        contrast = node.membership

        contrast = np.array(contrast * 1 / np.sum(contrast))

        target = contrast.dot(mu)

        # Naive after tree value
        # Confidence intervals
        CI = [contrast.dot(V) -
              np.linalg.norm(contrast) * sd_V * ndist.ppf(1 - level / 2),
              contrast.dot(V) +
              np.linalg.norm(contrast) * sd_V * ndist.ppf(1 - level / 2)]
        coverage.append((target >= CI[0] and target <= CI[1]))
        lengths.append(CI[1] - CI[0])

    if X_test is not None:
        pred = reg_tree.predict(X_test)
    else:
        pred = None

    return coverage, lengths, pred

# Replicating Figure 1

In [19]:
def terminal_inference_sim(n=50, p=5, a=0.1, b=0.1,
                           sd_y=1,
                           noise_sd_list=[0.5, 1, 2, 5],
                           UV_gamma_list=[],
                           use_nonrand=True,
                           start=0, end=100,
                           level=0.1, path=None):
    method_list = [f"RRT_{sd}" for sd in noise_sd_list]
    if use_nonrand:
        method_list += ["Tree val", "Naive"]
    for gamma in UV_gamma_list:
        method_list.append("UV_" + str(gamma))

    coverage_dict = {m: [] for m in method_list}
    length_dict = {m: [] for m in method_list}
    MSE_dict = {m: [] for m in method_list}

    for i in range(start, end):
        print(i, "th simulation")
        np.random.seed(i + 10000)
        X = np.random.normal(size=(n, p))

        mu = b * ((X[:, 0] <= 0) * (1 + a * (X[:, 1] > 0) + (X[:, 2] * X[:, 1] <= 0)))
        y = mu + np.random.normal(size=(n,), scale=sd_y)
        y_test = generate_test(mu, sd_y)
        
        if use_nonrand:
            # Tree value & naive inference & prediction
            (coverage_treeval, avg_len_treeval,
             coverage_treeval_naive, avg_len_treeval_naive,
             pred_test_treeval) = tree_values_inference(X, y, mu, sd_y=sd_y,
                                                        X_test=X, max_depth=1)
            MSE_test_treeval = (np.mean((y_test - pred_test_treeval) ** 2))

            coverage_dict["Tree val"].append(coverage_treeval)
            length_dict["Tree val"].append(avg_len_treeval)
            MSE_dict["Tree val"].append(MSE_test_treeval)
            coverage_dict["Naive"].append(coverage_treeval_naive)
            length_dict["Naive"].append(avg_len_treeval_naive)
            MSE_dict["Naive"].append(MSE_test_treeval)

        for noise_sd in noise_sd_list:
            # Create and train the regression tree
            reg_tree = RegressionTree(min_samples_split=50, max_depth=1,
                                      min_proportion=0., min_bucket=20)

            reg_tree.fit(X, y, sd=noise_sd * sd_y)

            coverage_i, lengths_i = randomized_inference(reg_tree=reg_tree,
                                                         y=y, sd_y=sd_y, mu=mu,
                                                         level=level)
            pred_test = reg_tree.predict(X)
            MSE_test = (np.mean((y_test - pred_test) ** 2))
            # Record results
            coverage_dict[f"RRT_{noise_sd}"].append(np.mean(coverage_i))
            length_dict[f"RRT_{noise_sd}"].append(np.mean(lengths_i))
            MSE_dict[f"RRT_{noise_sd}"].append(MSE_test)

        for gamma in UV_gamma_list:
            gamma_key = "UV_" + str(gamma)
            # UV decomposition
            coverage_UV, len_UV, pred_UV = UV_decomposition(X, y, mu, sd_y, X_test=X,
                                                            min_prop=0., max_depth=1,
                                                            min_sample=50, min_bucket=20,
                                                            gamma=gamma)
            MSE_UV = (np.mean((y_test - pred_UV) ** 2))
            coverage_dict[gamma_key].append(np.mean(coverage_UV))
            length_dict[gamma_key].append(np.mean(len_UV))
            MSE_dict[gamma_key].append(MSE_UV)

        if path is not None:
            joblib.dump([coverage_dict, length_dict, MSE_dict], path, compress=1)

    return coverage_dict, length_dict, MSE_dict

In [20]:
coverage_dict_fig1, length_dict_fig1, MSE_dict_fig1\
    = terminal_inference_sim(n=200, p=5, a=1, b=2,
                             sd_y=2,
                             noise_sd_list=[1, 2.5, 5, 10],
                             UV_gamma_list=[],
                             use_nonrand=True,
                             start=0, end=3,
                             level=0.1, path=None)

0 th simulation


/Users/yilingh/Desktop/PhD/SI_Codes/SI-CART/Utils/discrete_family.py:148: RuntimeWarning: overflow encountered in exp
  self._partition *= np.exp(_largest)


1 th simulation
2 th simulation


In [21]:
# Columns: 
# RRT_c: RRT with external randomization N(0, (c*sd_y)^2)
# Tree val: Tree-values
# Naive: naive inference
# Rows: Each row correspond to one round of simulation
pd.DataFrame(coverage_dict_fig1)

,RRT_1,RRT_2.5,RRT_5,RRT_10,Tree val,Naive
0,0.5,1.0,0.5,0.0,1.0,1.0
1,0.5,1.0,1.0,0.5,0.5,0.5
2,0.5,0.5,1.0,0.5,1.0,1.0


In [22]:
# Columns: 
# RRT_c: RRT with external randomization N(0, (c*sd_y)^2)
# Tree val: Tree-values
# Naive: naive inference
# Rows: Each row correspond to one round of simulation
pd.DataFrame(length_dict_fig1)

,RRT_1,RRT_2.5,RRT_5,RRT_10,Tree val,Naive
0,3.705403,2.339196,1.502620,1.125419,0.812268,0.660427
1,3.087178,20.627197,93.416963,20.155221,0.657966,0.657966
2,3.507186,2.276292,1.399645,6.690803,0.658560,0.658559


In [23]:
# Columns: 
# RRT_c: RRT with external randomization N(0, (c*sd_y)^2)
# Tree val: Tree-values
# Naive: naive inference
# Rows: Each row correspond to one round of simulation
pd.DataFrame(MSE_dict_fig1)

,RRT_1,RRT_2.5,RRT_5,RRT_10,Tree val,Naive
0,6.446936,6.622531,6.290958,6.494756,6.446936,6.446936
1,5.524521,5.301049,5.429078,5.301049,5.301049,5.301049
2,4.984089,5.028636,5.219642,4.955172,4.984089,4.984089


# Replicating Figure 2

In [14]:
coverage_dict_fig2, length_dict_fig2, MSE_dict_fig2\
    = terminal_inference_sim(n=200, p=10, a=1, b=2,
                             sd_y=2,
                             noise_sd_list=[5],
                             UV_gamma_list=[0.1, 0.2, 0.3, 0.4, 0.5],
                             use_nonrand=False,
                             start=0, end=10,
                             level=0.1, path=None)

0 th simulation


/Users/yilingh/Desktop/PhD/SI_Codes/SI-CART/Utils/discrete_family.py:148: RuntimeWarning: overflow encountered in exp
  self._partition *= np.exp(_largest)


1 th simulation
2 th simulation
3 th simulation
4 th simulation
5 th simulation
6 th simulation
7 th simulation
8 th simulation
9 th simulation


In [15]:
# Columns: 
# RRT_c: RRT with external randomization N(0, (c*sd_y)^2)
# UV_k: UV decomposition with gamma = k
# Rows: Each row correspond to one round of simulation
pd.DataFrame(coverage_dict_fig2)

,RRT_5,UV_0.1,UV_0.2,UV_0.3,UV_0.4,UV_0.5
0,0.5,0.5,0.5,1.0,1.0,1.0
1,0.0,0.5,1.0,1.0,0.5,1.0
2,0.5,1.0,0.5,1.0,1.0,1.0
3,0.5,1.0,1.0,1.0,1.0,1.0
4,0.0,1.0,1.0,1.0,1.0,1.0
5,0.5,1.0,1.0,1.0,1.0,1.0
6,0.0,0.5,1.0,1.0,1.0,0.5
7,0.0,0.5,1.0,0.5,1.0,1.0
8,0.5,0.5,1.0,1.0,1.0,0.5
9,1.0,1.0,1.0,1.0,1.0,1.0


In [16]:
# Columns: 
# RRT_c: RRT with external randomization N(0, (c*sd_y)^2)
# UV_k: UV decomposition with gamma = k
# Rows: Each row correspond to one round of simulation
pd.DataFrame(length_dict_fig2)

,RRT_5,UV_0.1,UV_0.2,UV_0.3,UV_0.4,UV_0.5
0,1.558794,2.187407,1.615507,1.372917,1.233864,1.141131
1,1.126128,2.185099,1.613802,1.371468,1.231635,1.141131
2,1.127882,2.182472,1.612589,1.369820,1.230896,1.139759
3,1.198107,2.190388,1.612589,1.374788,1.238794,1.139759
4,1.224475,2.198417,1.623638,1.379827,1.240074,1.148086
5,1.159793,2.188813,1.616545,1.372140,1.233166,1.139588
6,1.201420,2.182227,1.611681,1.369666,1.230942,1.139631
7,1.354658,2.182145,1.611621,1.369614,1.230942,1.139588
8,1.080272,2.184194,1.612589,1.370901,1.231635,1.140658
9,1.150893,2.182472,1.611863,1.369820,1.231080,1.139759


In [25]:
# Columns: 
# RRT_c: RRT with external randomization N(0, (c*sd_y)^2)
# UV_k: UV decomposition with gamma = k
# Rows: Each row correspond to one round of simulation
pd.DataFrame(MSE_dict_fig2)

,RRT_5,UV_0.1,UV_0.2,UV_0.3,UV_0.4,UV_0.5
0,5.566277,6.197180,5.947831,6.368900,6.460045,5.683047
1,4.974627,5.633572,5.921626,4.880301,5.802277,5.294574
2,5.499563,5.056115,4.981590,4.379431,5.041790,4.714836
3,5.225961,6.353509,5.849111,5.326090,6.001651,6.662719
4,3.841301,4.041951,3.979362,3.910499,4.292812,4.210228
5,6.383730,5.062984,5.509599,5.198781,5.320254,5.312965
6,5.153889,5.199020,5.191249,5.203030,4.993004,4.987964
7,5.384967,4.792183,4.881619,5.062421,5.060985,5.218473
8,5.463331,5.692806,5.748467,5.602949,6.097488,5.883444
9,5.073964,5.342965,5.490840,5.407444,5.172248,4.747513


In [19]:
cov = np.eye(5) + np.ones((5,5))

In [20]:
L = np.linalg.cholesky(np.linalg.inv(cov)).T

In [21]:
X = np.abs(np.random.normal(size=(5,100000)))

In [24]:
L

array([[ 0.91287093, -0.18257419, -0.18257419, -0.18257419, -0.18257419],
       [ 0.        ,  0.89442719, -0.2236068 , -0.2236068 , -0.2236068 ],
       [ 0.        ,  0.        ,  0.8660254 , -0.28867513, -0.28867513],
       [ 0.        ,  0.        ,  0.        ,  0.81649658, -0.40824829],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.70710678]])

In [25]:
np.min(L @ X)

np.float64(-1.6623417197792252)